In [2]:
import os
import cv2
import albumentations as A
import numpy as np

# Directorios
images_dir = r'dataset/images'
labels_dir = r'dataset/labels'
aug_images_dir = r'dataset/images_aumented'
aug_labels_dir = r'dataset/labels_aumented'

# Crear directorios si no existen
os.makedirs(aug_images_dir, exist_ok=True)
os.makedirs(aug_labels_dir, exist_ok=True)

# Función para cargar las imágenes
def load_images_and_labels(images_dir, labels_dir):
    images = []
    labels = []
    for filename in os.listdir(images_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(images_dir, filename)
            label_path = os.path.join(labels_dir, filename.replace('.jpg', '.txt').replace('.png', '.txt'))
            image = cv2.imread(img_path)
            if image is not None:
                images.append(image)
                with open(label_path, 'r') as f:
                    labels.append(f.read())
    return images, labels

# Función para guardar las imágenes aumentadas y las etiquetas
def save_augmented_data(image, label, image_name, label_name, aug_images_dir, aug_labels_dir):
    cv2.imwrite(os.path.join(aug_images_dir, image_name), image)
    with open(os.path.join(aug_labels_dir, label_name), 'w') as f:
        f.write(label)

# Definir las transformaciones de Albumentations
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=10, p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Función para aplicar las transformaciones y ajustar las etiquetas
def augment_data(image, label):
    # Parsear la etiqueta YOLO
    boxes = []
    class_labels = []
    for line in label.split('\n'):
        if line.strip() == '':
            continue
        parts = line.strip().split()
        class_labels.append(int(parts[0]))
        boxes.append([float(x) for x in parts[1:]])

    # Convertir a formato de Albumentations
    bboxes = []
    for box in boxes:
        bboxes.append(box)

    # Aplicar transformaciones
    augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    aug_image = augmented['image']
    aug_bboxes = augmented['bboxes']
    aug_class_labels = augmented['class_labels']

    # Asegurarse de que las coordenadas de las bounding boxes estén dentro del rango [0.0, 1.0]
    aug_bboxes = [[max(0, min(1, coord)) for coord in bbox] for bbox in aug_bboxes]

    # Convertir de vuelta a formato YOLO
    aug_label = ''
    for bbox, cls in zip(aug_bboxes, aug_class_labels):
        aug_label += f"{cls} " + " ".join(map(str, bbox)) + '\n'

    return aug_image, aug_label.strip()

# Cargar imágenes y etiquetas
images, labels = load_images_and_labels(images_dir, labels_dir)

# Aplicar las transformaciones y guardar los resultados
for i, (image, label) in enumerate(zip(images, labels)):
    for j in range(5):  # Generar 5 versiones aumentadas por imagen
        try:
            aug_image, aug_label = augment_data(image, label)
            image_name = f'aug_{i}_{j}.jpg'
            label_name = f'aug_{i}_{j}.txt'
            save_augmented_data(aug_image, aug_label, image_name, label_name, aug_images_dir, aug_labels_dir)
        except ValueError as e:
            print(f"Error al aumentar la imagen {i} versión {j}: {e}")
            continue


Error al aumentar la imagen 14 versión 0: Expected x_min for bbox (-0.07688121890547261, 0.20219216417910446, 0.11450559701492538, 0.6065764925373134, 0.191386815920398, 0.6282649253731343, 0.018812189054726383, 0.6282649253731343, 27) to be in the range [0.0, 1.0], got -0.07688121890547261.
Error al aumentar la imagen 14 versión 1: Expected x_min for bbox (-0.07688121890547261, 0.20219216417910446, 0.11450559701492538, 0.6065764925373134, 0.191386815920398, 0.6282649253731343, 0.018812189054726383, 0.6282649253731343, 27) to be in the range [0.0, 1.0], got -0.07688121890547261.
Error al aumentar la imagen 14 versión 2: Expected x_min for bbox (-0.07688121890547261, 0.20219216417910446, 0.11450559701492538, 0.6065764925373134, 0.191386815920398, 0.6282649253731343, 0.018812189054726383, 0.6282649253731343, 27) to be in the range [0.0, 1.0], got -0.07688121890547261.
Error al aumentar la imagen 14 versión 3: Expected x_min for bbox (-0.07688121890547261, 0.20219216417910446, 0.114505597

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 2.3 MB/s eta 0:00:00:00:0100:01


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Directorios
images_dir = 'dataset\images_aumented'
labels_dir = 'dataset\labels_aumented'
train_images_dir = r'dataset/yolov8/train/images'
train_labels_dir = r'dataset/yolov8/train/labels'
val_images_dir = r'dataset/yolov8/val/images'
val_labels_dir = r'dataset/yolov8/val/labels'

# Crear directorios si no existen
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Obtener lista de imágenes y etiquetas
images = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')]
labels = [f.replace('.jpg', '.txt').replace('.png', '.txt') for f in images]

# Dividir en entrenamiento y validación
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Mover archivos a los directorios correspondientes
def move_files(file_list, source_dir, dest_dir):
    for file_name in file_list:
        shutil.move(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

move_files(train_images, images_dir, train_images_dir)
move_files(val_images, images_dir, val_images_dir)
move_files(train_labels, labels_dir, train_labels_dir)
move_files(val_labels, labels_dir, val_labels_dir)


<>:6: SyntaxWarning: invalid escape sequence '\i'
<>:7: SyntaxWarning: invalid escape sequence '\l'
<>:6: SyntaxWarning: invalid escape sequence '\i'
<>:7: SyntaxWarning: invalid escape sequence '\l'
C:\Users\tomas\AppData\Local\Temp\ipykernel_9272\2069139183.py:6: SyntaxWarning: invalid escape sequence '\i'
  images_dir = 'dataset\images_aumented'
C:\Users\tomas\AppData\Local\Temp\ipykernel_9272\2069139183.py:7: SyntaxWarning: invalid escape sequence '\l'
  labels_dir = 'dataset\labels_aumented'


In [5]:
import ultralytics
from ultralytics import YOLO

# Carga el modelo preentrenado YOLOv8
model = YOLO('150_epochs.pt')  # Puedes usar 'yolov8s.pt' para un modelo más grande

# Configuración del entrenamiento
model.train(data='/kaggle/input/tp-cv2/dataset_kaggle.yaml',  # Ruta a tu archivo de configuración de datos
            epochs=150,  # Número de épocas
            imgsz=640,  # Tamaño de las imágenes
            batch=32,  # Tamaño del lote
            name='yolov8_envido')  # Nombre del proyecto

# Evalúa el modelo en el conjunto de validación
results = model.val()

# Guarda el modelo entrenado
model.save('/kaggle/working/yolov8_cards.pt')


Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/input/tp-cv2/dataset_kaggle.yaml, epochs=150, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8_envido2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


...

                   all        778       4677      0.711       0.66      0.737      0.483

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     15/150      4.95G      1.106      1.627      1.271        116        640: 100%|██████████| 98/98 [00:32<00:00,  3.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:05<00:00,  2.34it/s]


                   all        778       4677      0.743      0.645      0.733      0.472

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     16/150      4.74G      1.116      1.601      1.279        310        640:  49%|████▉     | 48/98 [00:15<00:17,  2.79it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
     16/150      4.74G      1.116      1.601      1.279        310        640:  49%|████▉     | 48/98 [00:16<00:16,  2.96it/s]


KeyboardInterrupt: 

In [ ]:
import torch
from IPython.display import Image, display

# Importar detect.py de YOLOv5
from pathlib import Path
detect_path = Path('/kaggle/working/yolov5/detect.py')
import detect as detect_path
TensorFlow.keras.utils

In [ ]:
!python /kaggle/working/yolov5/detect.py --source "/kaggle/input/tp-cv2/images_aumented/0baebe205cc430a7c752706599ab95fe.png"  --weights "/kaggle/working/yolov5/runs/train/yolo_training2/weights/best.pt" --conf 0.4